<a href="https://colab.research.google.com/github/VittorioBartolomeoSecondin/DVIS-CAValli_Team/blob/main/Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and connecting to Google Drive

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import re
import json
import csv
import requests
import os
import zipfile
from google.colab import files
from google.colab import drive
drive.mount('/content/drive') # turned out to be a good workaround to load a huge amount of data and keep it available

Mounted at /content/drive


# Load tree_dataset

Uncomment the following line of code to load the already pre-processed *tree_dataset* from google drive

In [ ]:
tree_dataset = pd.read_csv("/content/drive/MyDrive/tree_dataset/final_dataset/tree_dataset.csv", low_memory=False)

# Prepare data for mapping

In [ ]:
def create_abundance_csv(csv_name, header, name, abundance):

  with open(csv_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for idx in range(len(state)):
      row = [name[idx], abundance[idx]]
      writer.writerow(row)

In [ ]:
def create_abundance_fips_csv(csv_name, header, name, abundance, fips_codes):

  with open(csv_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for idx in range(len(state)):
      row = [name[idx], abundance[idx], fips_codes.get(name[idx])]
      writer.writerow(row)

## State abundance

In [ ]:
csv_name = "state_abundance.csv"
header = ['state', 'abundance']
state = list(tree_dataset['state'].value_counts().index)
abundance = list(tree_dataset['state'].value_counts().values)
create_abundance_csv(csv_name, header, state, abundance)

## State abundance and fips code

In [ ]:
fips_codes = {
    "Alabama": "US01",
    "Alaska": "US02",
    "Arizona": "US04",
    "Arkansas": "US05",
    "California": "US06",
    "Colorado": "US08",
    "Connecticut": "US09",
    "Delaware": "US10",
    "District of Columbia": "US11",
    "Florida": "US12",
    "Georgia": "US13",
    "Hawaii": "US15",
    "Idaho": "US16",
    "Illinois": "US17",
    "Indiana": "US18",
    "Iowa": "US19",
    "Kansas": "US20",
    "Kentucky": "US21",
    "Louisiana": "US22",
    "Maine": "US23",
    "Maryland": "US24",
    "Massachussetts": "US25",
    "Michigan": "US26",
    "Minnesota": "US27",
    "Mississippi": "US28",
    "Missouri": "US29",
    "Montana": "US30",
    "Nebraska": "US31",
    "Nevada": "US32",
    "New Hampshire": "US33",
    "New Jersey": "US34",
    "New Mexico": "US35",
    "New York": "US36",
    "North Carolina": "US37",
    "North Dakota": "US38",
    "Ohio": "US39",
    "Oklahoma": "US40",
    "Oregon": "US41",
    "Pennsylvania": "US42",
    "Rhode Island": "US44",
    "South Carolina": "US45",
    "South Dakota": "US46",
    "Tennessee": "US47",
    "Texas": "US48",
    "Utah": "US49",
    "Vermont": "US50",
    "Virginia": "US51",
    "Washington": "US53",
    "West Virginia": "US54",
    "Wisconsin": "US55",
    "Wyoming": "US56"
}

In [ ]:
csv_name = "state_abundance_fips.csv"
header = ['state', 'abundance', 'fips']
state = list(tree_dataset['state'].value_counts().index)
abundance = list(tree_dataset['state'].value_counts().values)
create_abundance_fips_csv(csv_name, header, state, abundance, fips_codes)

## State abundance and postal

In [ ]:
postal_codes = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "District of Columbia": "DC",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Puerto Rico": "PR",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Virgin Islands": "VI",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

In [ ]:
csv_name = "state_abundance_postal.csv"
header = ['state', 'abundance', 'postal']
state = list(tree_dataset['state'].value_counts().index)
abundance = list(tree_dataset['state'].value_counts().values)
create_abundance_fips_csv(csv_name, header, state, abundance, postal_codes)

## City abundance

In [ ]:
csv_name = "city_abundance.csv"
header = ['city', 'abundance']
city = list(tree_dataset['greater_metro'].value_counts().index)
abundance = list(tree_dataset['greater_metro'].value_counts().values)
create_abundance_csv(csv_name, header, city, abundance)

## JSON

In [ ]:
abundance_df = pd.read_csv("/content/state_abundance_postal.csv", low_memory=False)
abundance_dict = {}

for idx in range(abundance_df.shape[0]):
  name = list(abundance_df.iloc[idx])[0]
  abundance = list(abundance_df.iloc[idx])[1]
  postal = list(abundance_df.iloc[idx])[2]
  abundance_dict[postal] = [name, abundance]

for key, value in postal_codes.items():
  if abundance_dict.get(value) is None:
    name = key
    abundance = 0
    abundance_dict[value] = [name, abundance]
  else:
    name = abundance_dict.get(value)[0]
    abundance = abundance_dict.get(value)[1]

In [ ]:
json_path = "/content/drive/MyDrive/tree_dataset/final_dataset/us-states.json"
with open(json_path, 'r') as file:
    json_file = json.load(file)

In [ ]:
for idx in range(len(json_file['objects']['states'].get('geometries'))):
  properties = json_file['objects']['states'].get('geometries')[idx]['properties']
  postal = properties['postal']
  try:
    name, abundance = abundance_dict[postal]
    properties['name'] = name
    properties['abundance'] = str(abundance)
    print(properties)
  except:
    pass

{'postal': 'AK', 'name': 'Alaska', 'abundance': '0'}
{'postal': 'AL', 'name': 'Alabama', 'abundance': '0'}
{'postal': 'AR', 'name': 'Arkansas', 'abundance': '0'}
{'postal': 'AZ', 'name': 'Arizona', 'abundance': '214'}
{'postal': 'CA', 'name': 'California', 'abundance': '1806085'}
{'postal': 'CO', 'name': 'Colorado', 'abundance': '354878'}
{'postal': 'CT', 'name': 'Connecticut', 'abundance': '0'}
{'postal': 'DC', 'name': 'District of Columbia', 'abundance': '175847'}
{'postal': 'DE', 'name': 'Delaware', 'abundance': '0'}
{'postal': 'FL', 'name': 'Florida', 'abundance': '124208'}
{'postal': 'GA', 'name': 'Georgia', 'abundance': '41175'}
{'postal': 'HI', 'name': 'Hawaii', 'abundance': '15150'}
{'postal': 'IA', 'name': 'Iowa', 'abundance': '15805'}
{'postal': 'ID', 'name': 'Idaho', 'abundance': '0'}
{'postal': 'IL', 'name': 'Illinois', 'abundance': '0'}
{'postal': 'IN', 'name': 'Indiana', 'abundance': '97227'}
{'postal': 'KS', 'name': 'Kansas', 'abundance': '44767'}
{'postal': 'KY', 'name'

In [ ]:
out_path = "/content/choropleth.json"
with open(out_path, 'w') as file:
    json.dump(json_file, file)

## Drop data with no longitude/latitude

In [ ]:
# tree_dataset.dropna(subset=['longitude_coordinate', 'latitude_coordinate'], how='any', inplace=True)

# Download ALL .csv files

In [ ]:
directory_path = '/content'
file_type = ".csv"
files_to_download = glob.glob(f"{directory_path}/*{file_type}")
zip_filename = "datasets.zip"
with zipfile.ZipFile(zip_filename, "w") as zipf:
    for file in files_to_download:
        zipf.write(file, arcname=os.path.basename(file))
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>